# AUHack 2023 - Grundfos Hands-on ML Workshop - Building Type Classification
The goal of this workshop is to introduce you to how and what real-world data science is.

This workshop is based on an internal Data Hackthon, where the goal was to classify the type of building based on iGRID heat meter data.

As this is a hands-on workshop .....

Something about pragmatic => lost of exploring not too much ML.

In [1]:
# imports
import pandas as pd
import numpy as np

# database connector
import snowflake.connector

# tidyverse tools
import plotnine as p9
from plotnine import ggplot, aes, geom_point, geom_histogram, facet_wrap, theme, ggtitle

import patchworklib as pw

import siuba
from siuba import _, select, mutate, group_by, ungroup, filter, summarize
from siuba.dply.vector import lead, lag

# todo: where should the imports be? It feels bad to have them all here, but it also feels cluttered to have the all over the place?

No module named 'seaborn'


<Figure size 100x100 with 0 Axes>

In [ ]:
pd.set_option('display.width', 120)

p9.options.set_option('dpi', 300)
p9.options.set_option('figure_size', (8, 6))
# todo: test with codespaces

## Loading the data

We load the data from a Snowflake database using the `snowflake-connector-python` package and its `fetch_pandas_all()` function.

In [ ]:
snowflake_user = ''
snowflake_password = ''

conn = snowflake.connector.connect(
    account='da84422.west-europe.azure',
    #account='iw47870.west-europe.azure',
    user=snowflake_user,
    password=snowflake_password,
    database='GF_PROD_DB',
    schema='CURATED_HACKATHON',
    #warehouse='DATA_SCIENTIST_WH',
    #role='GRP_DATA_SCIENTIST_ROLE',
    )

cur = conn.cursor()
try:
    cur.execute("select * from GF_PROD_DB.CURATED_HACKATHON.V_DATA;")
    heat_data=cur.fetch_pandas_all()
    print('data:')
    print(heat_data.head(3))
    cur.execute("select * from GF_PROD_DB.CURATED_HACKATHON.V_METADATA;")
    metadata=cur.fetch_pandas_all()
    print('\nMetadata:')
    print(metadata.head(3))
finally:
    cur.close()
conn.close()

In [ ]:
# EXERCISE 1 SOLUTION
heat_data.info()
metadata.info()
print(f'\nLOCATION_ELEVATION is missing for {len(metadata) - metadata.LOCATION_ELEVATION.nunique()} out of {len(metadata)} buildings.')

In [ ]:
# Let's merge the heat meter data and metadata
# NOTE: In the real world, I would also split data here, but for simplicity we do that later.
data = metadata.merge(heat_data, how='left', on='METER_ID')

## Data engineering & exploration: Engineering part
The goal of this section is to give an introduction to the first steps a Data Scientist takes to understand new data. We want the data to be understandable, usable, and trustworthy.

In [ ]:
# select a meter_id to look at
meter_id_to_plot = metadata.sample(n=1)['METER_ID'].iloc[0]
print(f'We are investigating METER_ID = {meter_id_to_plot}')

In [ ]:
# we need to select a single METER_ID to get a comprehensible plot.
data_to_plot = (data >> filter(_.METER_ID == meter_id_to_plot)).reset_index()

# Let's have a look at the four values in df_train_meter
p1 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='ENERGY')) + geom_point())
p2 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='VOLUME')) + geom_point())
p3 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='FORWARD_TEMPERATURE_CUMULATIVE')) + geom_point())
p4 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='RETURN_TEMPERATURE_CUMULATIVE')) + geom_point())
p = (p1 | p2) / (p3 | p4)
p.savefig() # This is an artifact from Patchworklib, and does not save anything.

In [ ]:
# They are all cumulative, so let's convert energy, volume and the temperatures to delta values.
data = (
    data >>
    group_by('METER_ID') >>
    mutate(
        TIME_DELTA = _.TIMESTAMP - lag(_.TIMESTAMP, n=1, default=np.NaN),
        ENERGY_DELTA = _.ENERGY - lag(_.ENERGY, n=1, default=None),
        VOLUME_DELTA = _.VOLUME - lag(_.VOLUME, n=1, default=None),
        FORWARD_TEMPERATURE_DELTA = _.FORWARD_TEMPERATURE_CUMULATIVE - lag(_.FORWARD_TEMPERATURE_CUMULATIVE, n=1, default=None),
        RETURN_TEMPERATURE_DELTA = _.RETURN_TEMPERATURE_CUMULATIVE - lag(_.RETURN_TEMPERATURE_CUMULATIVE, n=1, default=None)
    ) >>
    ungroup()
)

print(data.head(5))

In [ ]:
# we need to select a single METER_ID to get a comprehensible plot.
data_to_plot = (data >> filter(_.METER_ID == meter_id_to_plot)).reset_index()

# Let's have a look at the four values in df_train_meter
p1 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='ENERGY_DELTA')) + geom_point())
p2 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='VOLUME_DELTA')) + geom_point())
p3 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='FORWARD_TEMPERATURE_DELTA')) + geom_point())
p4 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='RETURN_TEMPERATURE_DELTA')) + geom_point())
p = (p1 | p2) / (p3 | p4)
p.savefig() # This is an artifact from Patchworklib, and does not save anything.

In [ ]:
# The measurements are *supposed* to be daily, but let's make a sanity check
(
    ggplot(data >> filter(-np.isnat(_.TIME_DELTA )), aes('TIME_DELTA')) + geom_histogram(bins=40, fill='#e66066', color='black') +
    p9.scale_y_log10() +
    ggtitle('Distribution of time between measurements')
)


In [ ]:
# lets make the daily estimates of each column.
data = (
    data >>
    mutate(
        ENERGY_DAILY = _.ENERGY_DELTA * (pd.Timedelta(hours=24) / _.TIME_DELTA),
        VOLUME_DAILY = _.VOLUME_DELTA * (pd.Timedelta(hours=24) / _.TIME_DELTA),
        FORWARD_TEMPERATURE_DAILY = _.FORWARD_TEMPERATURE_DELTA * (pd.Timedelta(hours=24) / _.TIME_DELTA),
        RETURN_TEMPERATURE_DAILY = _.RETURN_TEMPERATURE_DELTA * (pd.Timedelta(hours=24) / _.TIME_DELTA),
    )
)

print(data.head(5))

In [ ]:
# Let's compare the 'energy_delta' and the 'energy_daily'.
data_to_plot = (data >> filter(_.METER_ID == meter_id_to_plot)).reset_index()

p1 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='ENERGY_DELTA')) + geom_point() + ggtitle('This plot shows the non-normalised values'))
p2 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='ENERGY_DAILY')) + geom_point() + ggtitle('This plot shows the normalised values with respect to the time gap'))
p = (p1 | p2)
p.savefig()

In [ ]:
# ... and similar comparison for 'forward_temperature'.
p1 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='FORWARD_TEMPERATURE_DELTA')) + geom_point() + ggtitle('This plot shows the non-normalised values'))
p2 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='FORWARD_TEMPERATURE_DAILY')) + geom_point() + ggtitle('This plot shows the normalised values with respect to the time gap'))
p = (p1 | p2)
p.savefig()

In [ ]:
# lets convert forward and return temperatures to Celcius.
data = (
    data >>
    mutate(
        FORWARD_TEMPERATURE_CELCIUS_DAILY = _.FORWARD_TEMPERATURE_DAILY / _.VOLUME_DAILY,
        RETURN_TEMPERATURE_CELCIUS_DAILY = _.RETURN_TEMPERATURE_DAILY / _.VOLUME_DAILY,
    )
)

print(data.head(5))

In [ ]:
# The temperatures should probably be between 0 and 100 degress celcius. Lets make another sanity check
(
    ggplot(data) +
    geom_histogram(aes('FORWARD_TEMPERATURE_CELCIUS_DAILY'), bins=40, fill='red', color='black', alpha=0.6) +
    geom_histogram(aes('RETURN_TEMPERATURE_CELCIUS_DAILY'), bins=40, fill='blue', color='black', alpha=0.6) +
    p9.scale_y_log10() +
    ggtitle('Distribution of forward and return temperatures, respectively.')
)

In [ ]:
# EXERCISE 2:
# * Create a new column, `TEMPERATURE_DIFFERENCE_CELCIUS_DAILY`, which show the difference between the Forward and the Return temperature.
# * Create a plot showing `FORWARD_TEMPERATURE_CELCIUS_DAILY` in red and `RETURN_TEMPERATURE_CELCIUS_DAILY` in blue.
# HINT: Add two `geom_points()` to the same ggplot. See: https://plotnine.readthedocs.io/en/stable/generated/plotnine.geoms.geom_point.html
# * Create a plot showing the new column `TEMPERATURE_DIFFERENCE_CELCIUS_DAILY`.
# * Combine the two plots using Patchworklib.

In [ ]:
# EXERCISE 2 SOLUTION

# Let's calculate the temperature difference
data = (
    data >>
    mutate(TEMPERATURE_DIFFERENCE_CELCIUS_DAILY = _.FORWARD_TEMPERATURE_CELCIUS_DAILY - _.RETURN_TEMPERATURE_CELCIUS_DAILY)
)

# Let`s plot the three temperatures
data_to_plot = (data >> filter(_.METER_ID == meter_id_to_plot)).reset_index()

p1 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='ENERGY_DELTA')) + geom_point() + ggtitle('This plot shows the non-normalised values'))
p2 = pw.load_ggplot(ggplot(data_to_plot, aes(x='TIMESTAMP', y='ENERGY_DAILY')) + geom_point() + ggtitle('This plot shows the normalised values with respect to the time gap'))
p = (p1 | p2)
p.savefig()


p1 = pw.load_ggplot(
    ggplot(data_to_plot, aes(x='TIMESTAMP')) +
    geom_point(aes(y='FORWARD_TEMPERATURE_CELCIUS_DAILY'), color='red') +
    geom_point(aes(y='RETURN_TEMPERATURE_CELCIUS_DAILY'), color='blue') +
    theme(axis_text_x = p9.element_text(rotation=90, hjust=0.35)) +
    ggtitle('The forward (red) and return (blue) temperatures.')
)
p2 = pw.load_ggplot(
    ggplot(data_to_plot, aes(x='TIMESTAMP',y='TEMPERATURE_DIFFERENCE_CELCIUS_DAILY')) +
    geom_point() +
    theme(axis_text_x = p9.element_text(rotation=90, hjust=0.35)) +
    ggtitle('The difference between forward and return temperature.')
)
p = (p1 | p2)
p.savefig()

## Data engineering & exploration: Exploration part

In [ ]:
# Before considering features, let's see the distrubution of our target; `BUILDING_TYPE`
(
    ggplot(metadata, aes('BUILDING_TYPE', fill='BUILDING_TYPE')) +
    p9.geom_bar(color='black') +
    ggtitle('The counts of residential and non-residentail meters included in the dataset')
)

In [ ]:
# Let's consider the metadata features and start with temperature difference.
(
    ggplot(data, aes('TEMPERATURE_DIFFERENCE_CELCIUS_DAILY', fill='BUILDING_TYPE')) +
    p9.geom_histogram(bins=30, color='black') +
    p9.xlim(-5, 60) +
    ggtitle('The distribution of the temperature differences (for all meters)')
)

In [ ]:
# ... given the uneven distribution of residential and non-residential buildings, the above is hard to decipher. Lets split the plot.
(
    ggplot(data, aes('TEMPERATURE_DIFFERENCE_CELCIUS_DAILY', fill='BUILDING_TYPE')) +
    p9.geom_histogram(bins=30, color='black') +
    p9.xlim(-5, 60) +
    facet_wrap('~BUILDING_TYPE', scales='free_y') +
    ggtitle('The distribution of the temperature differences (for all meters)')
)

In [ ]:
# ... and repeat this for ENERGY_DAILY.
(
    ggplot(data, aes('ENERGY_DAILY', fill='BUILDING_TYPE')) +
    p9.geom_histogram(bins=100, color='black') +
    p9.xlim(0, 1) +
    facet_wrap('~BUILDING_TYPE', scales='free_y') +
    ggtitle('The distribution of the daily energy consumption (for all meters)')
)

In [ ]:
# Let's look at a couple of the metadata variables.
(
    ggplot(metadata, aes('BUILT_UPON_AREA', fill='BUILDING_TYPE')) +
    p9.geom_histogram(bins=40, color='black') +
    p9.xlim(0, 2000) +
    facet_wrap('~BUILDING_TYPE', scales='free_y') +
    ggtitle('The distribution of the built upon area (for all meters)')
)

In [ ]:
# EXERCISE 3:
# * Make a histogram plot of the `LOCATION_ELEVATION` split by `BUILDING_TYPE`
# * What does this result tell us?
# * Why is this a surprising signal (and what won't it generalize beyond this dataset).

In [ ]:
# EXERCISE 3 SOLUTION

# Let's look at the `LOCATION_ELEVATION`
(
    ggplot(metadata, aes('LOCATION_ELEVATION', fill='BUILDING_TYPE')) +
    p9.geom_histogram(bins=55, color='black') +
    p9.xlim(0, 55) +
    facet_wrap('~BUILDING_TYPE', scales='free_y') +
    ggtitle('The distribution of the built upon area (for all meters)')
)
# Answer: We can see that the non-residential building are closer to the sea level in this city. This is unlikely to generalize to other cities.

## Feature engineering
The goal of this section is extract features from the time series that can be used in our models.

At the end of this section we will have a pruned dataset ready to use for modeling.

In [ ]:
# Let's calculate some features from the daily energy usage
features = (
    data >>
    group_by('METER_ID') >>
    summarize(
        ENERGY_DAILY_MEAN = _.ENERGY_DAILY.mean(),
        ENERGY_DAILY_MEDIAN = _.ENERGY_DAILY.median(),
        ENERGY_DAILY_CV = _.ENERGY_DAILY.std() / _.ENERGY_DAILY.mean(),
        ENERGY_DAILY_AUTOCORR = _.ENERGY_DAILY.autocorr(),
    )
)

print(features)

In [ ]:
# Let's add the BUILDING_TYPE and visualize.
features = metadata.merge(features, how='left', on='METER_ID')
print(features)

In [ ]:
# Let's look at a couple of the metadata variables.
# NOTE: We use geom_density here as there are (relatively) few datapoints/rows.
p1 = pw.load_ggplot(
    ggplot(features, aes('ENERGY_DAILY_MEAN', fill='BUILDING_TYPE')) +
    p9.geom_density(alpha=0.5) +
    p9.xlim(0, 1.5) +
    ggtitle('The density of the Mean of the daily energy consumption')
)

p2 = pw.load_ggplot(
    ggplot(features, aes('ENERGY_DAILY_MEDIAN', fill='BUILDING_TYPE')) +
    p9.geom_density(alpha=0.5) +
    p9.xlim(0, 1.5) +
    ggtitle('The density of the Median of the daily energy consumption')
)

p3 = pw.load_ggplot(
    ggplot(features, aes('ENERGY_DAILY_CV', fill='BUILDING_TYPE')) +
    p9.geom_density(alpha=0.5) +
    p9.xlim(0, 4) +
    ggtitle('The density of the Coefficient of Variance of the daily energy consumption')
)

p4 = pw.load_ggplot(
    ggplot(features, aes('ENERGY_DAILY_AUTOCORR', fill='BUILDING_TYPE')) +
    p9.geom_density(alpha=0.5) +
    p9.xlim(0, 1) +
    ggtitle('The density of the Autocorrelation of the daily energy consumption')
)

p = (p1 | p2) / (p3 | p4)
p.savefig()

In [ ]:
# EXERCISE 4:
# * Calculate `Mean` and `Coefficient of Variance` for daily Energy, Volume, Forward Temperature (celcius), Return Temperature (celcius), and Temperature Difference (celcius).
# HINT: overwrite the features DataFrame.
# * Calculate another feature!
# HINT: you can find inspiration here: https://pandas.pydata.org/docs/reference/api/pandas.Series.describe.html

In [ ]:
# EXERCISE 4 SOLUTION

from numpy import mean, absolute

features = (
    data >>
    group_by('METER_ID') >>
    summarize(
        ENERGY_DAILY_MEAN = _.ENERGY_DAILY.mean(),
        ENERGY_DAILY_MEDIAN = _.ENERGY_DAILY.median(),
        ENERGY_DAILY_CV = _.ENERGY_DAILY.std() / _.ENERGY_DAILY.mean(),
        ENERGY_DAILY_AUTOCORR = _.ENERGY_DAILY.autocorr(),
        ENERGY_DAILY_MAD = mean(absolute(_.ENERGY_DAILY - mean(_.ENERGY_DAILY))),
        VOLUME_DAILY_MEAN = _.VOLUME_DAILY.mean(),
        VOLUME_DAILY_CV = _.VOLUME_DAILY.std() / _.VOLUME_DAILY.mean(),
        FORWARD_TEMPERATURE_CELCIUS_DAILY_MEAN = _.FORWARD_TEMPERATURE_CELCIUS_DAILY.mean(),
        FORWARD_TEMPERATURE_CELCIUS_DAILY_CV = _.FORWARD_TEMPERATURE_CELCIUS_DAILY.std() / _.FORWARD_TEMPERATURE_CELCIUS_DAILY.mean(),
        RETURN_TEMPERATURE_CELCIUS_DAILY_MEAN = _.RETURN_TEMPERATURE_CELCIUS_DAILY.mean(),
        RETURN_TEMPERATURE_CELCIUS_DAILY_CV = _.RETURN_TEMPERATURE_CELCIUS_DAILY.std() / _.RETURN_TEMPERATURE_CELCIUS_DAILY.mean(),
        TEMPERATURE_DIFFERENCE_CELCIUS_DAILY_MEAN = _.TEMPERATURE_DIFFERENCE_CELCIUS_DAILY.mean(),
        TEMPERATURE_DIFFERENCE_CELCIUS_DAILY_CV = _.TEMPERATURE_DIFFERENCE_CELCIUS_DAILY.std() / _.TEMPERATURE_DIFFERENCE_CELCIUS_DAILY.mean(),
        ENERGY_DAILY_Q25 = _.ENERGY_DAILY.quantile(q=0.25),
        ENERGY_DAILY_Q75 = _.ENERGY_DAILY.quantile(q=0.75),
        TEMPERATURE_DIFFERENCE_CELCIUS_DAILY_MIN = _.TEMPERATURE_DIFFERENCE_CELCIUS_DAILY.min(),
        TEMPERATURE_DIFFERENCE_CELCIUS_DAILY_MAX = _.TEMPERATURE_DIFFERENCE_CELCIUS_DAILY.max(),
    )
)

print(features)

In [ ]:
# Let's create our dataset for modelling. First lets remove unnecessary columns.
data_final  = (
    metadata >>
    select(- _.contains('UNIT')) >>
    select(- _.TIMESTAMP_TIMEZONE) >>
    select(- _.METER_TYPE)
).merge(features, how='left', on='METER_ID')

data_final.info()

In [ ]:

data_final = data_final.fillna(0)
float64_cols = list(data_final.select_dtypes(include='float64'))
data_final[float64_cols] = data_final[float64_cols].astype('float32')

data_final.info()

## Modeling
The goal of this section is train and test a simple model to predict the BUILDING_TYPE!

In [ ]:
# First we split data into train and test
from sklearn.model_selection import train_test_split
metadata_slim = (
    metadata >>
    select(- _.contains('UNIT')) >>
    select(- _.TIMESTAMP_TIMEZONE) >>
    select(- _.METER_TYPE)
)

train, test = train_test_split(metadata_slim, test_size = 0.25, stratify=metadata_slim['BUILDING_TYPE'])
#train, test = train_test_split(data_final, test_size = 0.25, stratify=data_final['BUILDING_TYPE'])
print(f'Number of train examples: {len(train)} out of {len(data_final)}.')
print(f'Number of test examples: {len(test)} out of {len(data_final)}.')

X_train = (train >> select(- _.BUILDING_TYPE)).fillna(0).to_numpy()
y_train = (train >> select(_.BUILDING_TYPE)).fillna(0).to_numpy()
X_test = (test >> select(- _.BUILDING_TYPE)).fillna(0).to_numpy()
y_test = (test >> select(_.BUILDING_TYPE)).fillna(0).to_numpy()

In [ ]:
# Lets train a decision tree
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [ ]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)

In [ ]:
clf.predict(X_test)